In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [2]:
names = open('../data/names.txt', 'r').read().splitlines()
names[:4]

['emma', 'olivia', 'ava', 'isabella']

In [3]:
len(names)

32033

In [4]:
chars = sorted(set(''.join(names)))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
stoi.items()
itos = {i: s for s, i in stoi.items()}

In [5]:
block_size = 3

X = []
y = []

for name in names[:5]:
    
    context = [0] * block_size
    
    for ch in name + '.':
        
        idx = stoi[ch]
        
        X.append(context)
        y.append(idx)
        
        context = context[1:] + [idx]

X = torch.tensor(X)
y = torch.tensor(y)

print(X[:5])
print(y[:5])
    
        
        

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])
tensor([ 5, 13, 13,  1,  0])


In [6]:
emb_size = 2
C = torch.randn((27, emb_size))

emb = C[X]
emb.shape


torch.Size([32, 3, 2])

In [7]:
W1 = torch.randn(((block_size * emb_size), 100), requires_grad=True)
b1 = torch.randn(100, requires_grad=True)

In [8]:
h = torch.tanh(emb.view(emb.shape[0],(emb.shape[1] * emb.shape[2])) @ W1 + b1)
h.shape

torch.Size([32, 100])

In [9]:
W2 = torch.randn((100, 27), requires_grad = True)
b2 = torch.randn(27, requires_grad = True)
parameters = [C, W1, b1, W2, b2]
sum(p.nelement() for p in parameters)

3481

In [10]:
for p in parameters:
    p.requires_grad = True

In [11]:
logits = h @ W2 + b2
logits.shape

counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)

loss = -prob[torch.arange(emb.shape[0]), y].log().mean()
loss

tensor(15.6745, grad_fn=<NegBackward0>)

In [12]:
logits = h @ W2 + b2
logits.shape

loss = F.cross_entropy(logits, y)

loss

tensor(15.6745, grad_fn=<NllLossBackward0>)

In [ ]:
for _ in range(10):
    #forward pass
    emb = C[X] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.1 * p.grad